# Import des librairies et chargement des données

In [2]:
import os

# grammaire abstraite de l'arbre syntaxique de Python
import ast

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import is_string_dtype, is_numeric_dtype

pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
dossier_data = "data/"
nom_fichier = "batiments_data.csv"
batiments_data = pd.read_csv(dossier_data+nom_fichier)

In [4]:
pd.set_option("display.max_columns", None)
batiments_data.head()

,OSEBuildingID,BuildingType,PrimaryPropertyType,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFABuilding,ENERGYSTARScore,SiteEnergyUse,SteamUse,Electricity,NaturalGas,OtherFuelUse,GHGEmissions,BuildingAge,HasUsedSteam,IsSteamUser,IsElectricityUser,IsNaturalGasUser,IsOtherFuelUser,NumberOfUsedEnergies,MostUsedEnergy,GatheredPrimaryPropertyType
0,3,NonResidential,Hotel,DOWNTOWN,1969,1.0,41.0,961990.0,18.0,73130656.0,19660404.0,49762435.0,3709900.0,0.0,2061.48,46,1.0,1.0,1.0,1.0,0.0,3.0,Electricity,Hotel
1,5,NonResidential,Hotel,DOWNTOWN,1926,1.0,10.0,61320.0,1.0,28229320.0,23458518.0,2769023.0,2001894.0,0.0,1936.34,89,1.0,1.0,1.0,1.0,0.0,3.0,Steam,Hotel
2,8,NonResidential,Hotel,DOWNTOWN,1980,1.0,18.0,107430.0,67.0,14829099.0,0.0,6066245.0,8763105.0,0.0,507.70,35,0.0,0.0,1.0,1.0,0.0,2.0,NaturalGas,Hotel
3,19,NonResidential,Hotel,DOWNTOWN,1922,1.0,11.0,67390.0,14.0,10711451.0,4403788.0,4089407.0,2218425.0,0.0,486.25,93,1.0,1.0,1.0,1.0,0.0,3.0,Steam,Hotel
4,25,NonResidential,Hotel,DOWNTOWN,1916,1.0,10.0,104352.0,83.0,7845112.0,3205497.0,1790665.0,2849024.0,0.0,411.22,99,1.0,1.0,1.0,1.0,0.0,3.0,Steam,Hotel


# Sélection des variables finales

In [5]:
batiments_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2322 entries, 0 to 2321
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   OSEBuildingID                2322 non-null   int64  
 1   BuildingType                 2322 non-null   object 
 2   PrimaryPropertyType          2322 non-null   object 
 3   Neighborhood                 2322 non-null   object 
 4   YearBuilt                    2322 non-null   int64  
 5   NumberofBuildings            2322 non-null   float64
 6   NumberofFloors               2322 non-null   float64
 7   PropertyGFABuilding          2322 non-null   float64
 8   ENERGYSTARScore              1532 non-null   float64
 9   SiteEnergyUse                2322 non-null   float64
 10  SteamUse                     2322 non-null   float64
 11  Electricity                  2322 non-null   float64
 12  NaturalGas                   2322 non-null   float64
 13  OtherFuelUse      

In [8]:
variables_ecartees = ["OSEBuildingID", "PrimaryPropertyType", "ENERGYSTARScore", "YearBuilt", "SteamUse", "Electricity",
                      "NaturalGas", "OtherFuelUse"]
batiments_data_modeles = batiments_data.drop(columns=variables_ecartees)

# Préparation des données

## Variables catégorielles

In [9]:
batiments_data_modeles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2322 entries, 0 to 2321
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   BuildingType                 2322 non-null   object 
 1   Neighborhood                 2322 non-null   object 
 2   NumberofBuildings            2322 non-null   float64
 3   NumberofFloors               2322 non-null   float64
 4   PropertyGFABuilding          2322 non-null   float64
 5   SiteEnergyUse                2322 non-null   float64
 6   GHGEmissions                 2322 non-null   float64
 7   BuildingAge                  2322 non-null   int64  
 8   HasUsedSteam                 2322 non-null   float64
 9   IsSteamUser                  2322 non-null   float64
 10  IsElectricityUser            2322 non-null   float64
 11  IsNaturalGasUser             2322 non-null   float64
 12  IsOtherFuelUser              2322 non-null   float64
 13  NumberOfUsedEnergi

In [ ]:
variables_categorielles = []